In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
    ElementClickInterceptedException,
    TimeoutException
)

########################################
# 기본 설정
########################################
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")

url = "https://map.naver.com/p/search/%EC%A4%91%EB%9E%91%EA%B5%AC%20%EC%83%81%EB%B4%89%EB%8F%99%20%EC%9D%BC%EC%8B%9D?c=15.00,0,0,0,dh"

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)
driver.get(url)

########################################
# 공통 유틸
########################################
def to_default():
    driver.switch_to.default_content()

def switch_to_search_iframe(timeout=20):
    to_default()
    WebDriverWait(driver, timeout).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe"))
    )
    close_popups(driver)  # ✅ 추가

def switch_to_entry_iframe(timeout=20):
    to_default()
    WebDriverWait(driver, timeout).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
    )
    close_popups(driver)  # ✅ 추가

def js_click(el):
    driver.execute_script("arguments[0].click();", el)

def safe_click(el):
    # ✅ 클릭 전에 팝업/광고 닫기
    close_popups(driver)

    try:
        el.click()
    except ElementClickInterceptedException:
        # ✅ 막혔으면 팝업 다시 닫고 JS 클릭
        close_popups(driver)
        js_click(el)
    except Exception:
        # 마지막 수단
        try:
            js_click(el)
        except Exception:
            pass

def scroll_center(el):
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    time.sleep(0.2)

def safe_text(css, default=""):
    try:
        els = driver.find_elements(By.CSS_SELECTOR, css)
        if els:
            return (els[0].text or "").strip()
    except Exception:
        pass
    return default

########################################
# 1번 코드: 현재 페이지 끝까지 스크롤
########################################
def scroll_list_to_end(pause=1.0, max_try=120):
    scroll_box = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div#_pcmap_list_scroll_container"))
    )

    last_height = 0
    same_count = 0

    for _ in range(max_try):
        height = driver.execute_script("return arguments[0].scrollHeight", scroll_box)
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_box)
        time.sleep(pause)

        if height == last_height:
            same_count += 1
        else:
            same_count = 0

        if same_count >= 3:
            break

        last_height = height

########################################
# 1번 코드: 하단 네비게이터에서 "마지막 페이지 버튼" 클릭
########################################
def click_last_page_button():
    """
    searchIframe 내부에서 하단 네비게이터의 마지막 a를 클릭.
    더 이상 버튼이 없으면 False
    """
    try:
        nav_container = driver.find_element(By.CSS_SELECTOR, "#app-root div.zRM9F")
        page_buttons = nav_container.find_elements(By.TAG_NAME, "a")
        if not page_buttons:
            return False

        last_btn = page_buttons[-1]
        driver.execute_script("arguments[0].scrollIntoView(true);", last_btn)
        time.sleep(0.4)
        safe_click(last_btn)
        return True

    except NoSuchElementException:
        return False
    except Exception:
        return False

########################################
# ✅ 리뷰 탭 클릭: nth-child 금지, '리뷰' 텍스트로 클릭
########################################
def click_review_tab_by_text(timeout=10):
    """
    place_fixed_maintab 안에서 '리뷰' 글자가 포함된 탭(a)을 찾아 클릭.
    가게마다 탭 순서가 달라도 안정적.
    """
    try:
        tab_area = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.place_fixed_maintab"))
        )
        tabs = tab_area.find_elements(By.CSS_SELECTOR, "a")

        for a in tabs:
            txt = (a.text or "").strip().replace("\n", " ")
            if "리뷰" in txt:
                scroll_center(a)
                safe_click(a)
                time.sleep(1.2)
                return True

        return False
    except TimeoutException:
        return False

def wait_review_list_loaded(timeout=12):
    end = time.time() + timeout
    while time.time() < end:
        try:
            if driver.find_elements(By.CSS_SELECTOR, "#_review_list > li"):
                return True
        except Exception:
            pass
        time.sleep(0.3)
    return False

########################################
# 2번 코드: 리뷰 '리스트 더보기' 버튼 4회 클릭 (원본 셀렉터 유지)
########################################
MORE_BTN_SELECTOR_SPAN = (
    "#app-root > div > div > div:nth-child(7) > div:nth-child(3) "
    "> div.place_section.k1QQ5 > div.NSTUp > div > a > span"
)

def click_review_list_more_fixed(times=4):
    """
    원본 셀렉터(span) 그대로 찾고, 실제 클릭은 부모 a를 클릭.
    """
    for i in range(times):
        switch_to_entry_iframe()
        try:
            span = WebDriverWait(driver, 8).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, MORE_BTN_SELECTOR_SPAN))
            )
            a = span.find_element(By.XPATH, "./parent::a")

            before_cnt = len(driver.find_elements(By.CSS_SELECTOR, "#_review_list > li"))

            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", a)
            time.sleep(0.25)
            safe_click(a)
            time.sleep(1.3)

            after_cnt = len(driver.find_elements(By.CSS_SELECTOR, "#_review_list > li"))
            print(f"🔁 리뷰 리스트 더보기 {i+1}회: {before_cnt} -> {after_cnt}")

            if after_cnt <= before_cnt:
                print("⛔ 리뷰 수 증가 없음 → 더보기 종료")
                break

        except TimeoutException:
            print("⛔ 리뷰 리스트 더보기 버튼 없음 → 종료")
            break
        except StaleElementReferenceException:
            time.sleep(0.3)
            continue

########################################
# 2번 코드: 리뷰 수집 (원본 유지)
########################################
REVIEW_LI = "#_review_list > li"
REVIEW_WRAP = "div.pui__vn15t2"
READ_MORE_CANDIDATES = ["a.pui__jhpEy", "a[role='button']", "a"]

def scroll_under_header(el):
    driver.execute_script("arguments[0].scrollIntoView({block:'start'});", el)
    time.sleep(0.15)
    header_h = driver.execute_script("""
        const h = document.querySelector('#_header');
        return h ? h.getBoundingClientRect().height : 0;
    """)
    driver.execute_script("window.scrollBy(0, arguments[0] + 12);", header_h)
    time.sleep(0.15)

def click_read_more_if_exists(wrap):
    for sel in READ_MORE_CANDIDATES:
        try:
            anchors = wrap.find_elements(By.CSS_SELECTOR, sel)
        except Exception:
            anchors = []

        for a in anchors:
            try:
                txt = (a.text or "").strip()
                if "더보기" in txt:
                    scroll_under_header(a)
                    try:
                        a.click()
                    except ElementClickInterceptedException:
                        js_click(a)
                    time.sleep(0.2)
                    return True
            except StaleElementReferenceException:
                continue
    return False

def extract_review_text_from_wrap(wrap):
    raw = (wrap.text or "").strip()
    return raw.replace("더보기", "").strip()

def crawl_visible_reviews(wait_sec=10):
    switch_to_entry_iframe()
    wait_local = WebDriverWait(driver, wait_sec)
    wait_local.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, REVIEW_LI)))

    reviews = []
    li_elems = driver.find_elements(By.CSS_SELECTOR, REVIEW_LI)

    for idx in range(len(li_elems)):
        for _ in range(3):
            try:
                switch_to_entry_iframe()
                li = driver.find_elements(By.CSS_SELECTOR, REVIEW_LI)[idx]
                wrap = li.find_element(By.CSS_SELECTOR, REVIEW_WRAP)

                click_read_more_if_exists(wrap)
                text = extract_review_text_from_wrap(wrap)
                if text:
                    reviews.append(text)
                break
            except (StaleElementReferenceException, NoSuchElementException, IndexError):
                time.sleep(0.2)
                continue

    return reviews

########################################
# entryIframe에서 "정보 + 리뷰" 뽑는 함수
########################################
def scrape_current_place():
    switch_to_entry_iframe()
    time.sleep(1.0)

    # 상호/업종
    name = safe_text("#_title div span.GHAhO", "")
    category = safe_text("#_title > div > span.lnJFt", "")

    # 주소 펼치기 버튼 (원본 유지)
    try:
        add_btn = driver.find_element(
            By.CSS_SELECTOR,
            "#app-root > div > div > div:nth-child(6) div div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div a"
        )
        scroll_center(add_btn)
        safe_click(add_btn)
        time.sleep(1.2)
    except Exception:
        pass

    # 주소 (원본 유지)
    add1 = safe_text(
        "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) "
        "> div.place_section_content > div > div.O8qbU.tQY7D "
        "> div > div.Y31Sf > div:nth-child(1)", ""
    )
    add2 = safe_text(
        "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) "
        "> div.place_section_content > div > div.O8qbU.tQY7D "
        "> div > div.Y31Sf > div:nth-child(2)", ""
    )
    address = " ".join([x for x in [add1, add2] if x]).strip()

    # ✅ 리뷰 탭 클릭: 텍스트 기반
    reviews = []
    ok = click_review_tab_by_text()
    if ok:
        loaded = wait_review_list_loaded(timeout=12)
        if loaded:
            # 리스트 더보기 4회 (원본 셀렉터 유지)
            click_review_list_more_fixed(times=4)
            # 리뷰 수집
            try:
                reviews = crawl_visible_reviews()
            except Exception:
                reviews = []
        else:
            # 리뷰 탭은 눌렀는데 리뷰 리스트가 안 뜨는 케이스
            reviews = []
    else:
        # 리뷰 탭 자체가 없는 가게일 수도 있음
        reviews = []

    return {
        "name": name,
        "category": category,
        "address": address,
        "reviews": reviews
    }
########################################
def close_popups(driver):
    """
    네이버 지도에서 뜨는 팝업/광고 + '주문/포장 상품 상세' 같은 모달을 닫는 함수
    """
    try:
        # 0) ESC로 먼저 닫기(모달에 매우 잘 먹힘)
        try:
            driver.switch_to.active_element.send_keys(Keys.ESCAPE)
            time.sleep(0.2)
        except Exception:
            pass

        # 1) 좌상단 뒤로(←) / 닫기(X) 버튼 후보들 클릭
        #    (네 스샷처럼 좌상단에 '←'가 있는 케이스가 많음)
        close_selectors = [
            # aria-label 기반
            "button[aria-label='닫기']",
            "button[aria-label*='닫기']",
            "a[aria-label='닫기']",
            "a[aria-label*='닫기']",
            "button[aria-label='이전']",
            "button[aria-label*='이전']",
            "a[aria-label='이전']",
            "a[aria-label*='이전']",
            # class/role 기반 (보수적)
            "button[class*='close']",
            "a[class*='close']",
            "button[role='button'][class*='Close']",
            "button[role='button']",
            "a[role='button']",
        ]

        for sel in close_selectors:
            els = driver.find_elements(By.CSS_SELECTOR, sel)
            for el in els[:6]:
                try:
                    txt = (el.text or "").strip()
                    # 텍스트가 없을 수도 있지만, 있으면 닫기/이전/취소 계열만 우선
                    if txt and not any(k in txt for k in ["닫기", "이전", "취소", "뒤로", "X", "×", "←"]):
                        continue
                    driver.execute_script("arguments[0].click();", el)
                    time.sleep(0.25)
                except Exception:
                    pass

        # 2) 주문 모달에서 흔한 "주문서 담기" 같은 버튼이 보이면
        #    현재 화면이 모달/주문 상세일 확률이 높으니 "브라우저 back"도 한 번 시도
        try:
            if driver.find_elements(By.XPATH, "//*[contains(text(),'주문서 담기') or contains(text(),'미리 포장') or contains(text(),'주문')]"):
                driver.execute_script("window.history.back();")
                time.sleep(0.4)
        except Exception:
            pass

        # 3) 화면을 덮는 overlay/dim 제거(클릭 막힘 방지)
        overlay_selectors = [
            "div[class*='dim']",
            "div[class*='Dim']",
            "div[class*='overlay']",
            "div[class*='Overlay']",
            "div[role='dialog']",
        ]
        for sel in overlay_selectors:
            ovs = driver.find_elements(By.CSS_SELECTOR, sel)
            for ov in ovs[:3]:
                try:
                    driver.execute_script("arguments[0].remove();", ov)
                except Exception:
                    pass

    except Exception:
        pass
########################################
# 검색 결과 한 페이지에서 모든 카드(현재 로딩된 것) 순회
########################################
def get_visible_card_count():
    switch_to_search_iframe()
    return len(driver.find_elements(By.CSS_SELECTOR, "li.UEzoS"))

def click_card_by_index(idx):
    """
    searchIframe에서 idx번째 카드를 클릭하여 entryIframe으로 진입
    """
    switch_to_search_iframe()
    cards = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
    if idx >= len(cards):
        return False

    card = cards[idx]
    scroll_center(card)
    safe_click(card)
    time.sleep(1.5)
    return True

########################################
# 메인: 페이지들 돌면서 가게 전부 수집
########################################
results = []
visited = set()  # (name, address) 기반 중복 방지

page_round = 1
switch_to_search_iframe()

while True:
    switch_to_search_iframe()
    print(f"\n===== PAGE ROUND {page_round} =====")

    # 1) 현재 페이지 리스트 끝까지 스크롤해서 카드 전부 로딩
    scroll_list_to_end()
    time.sleep(0.6)

    # 2) 현재 페이지 카드 개수 확정
    card_count = get_visible_card_count()
    print(f"📦 현재 페이지 카드 수: {card_count}")

    # 3) 카드 하나씩 클릭 -> entryIframe에서 정보/리뷰 수집
    for i in range(card_count):
        try:
            ok = click_card_by_index(i)
            if not ok:
                continue

            data = scrape_current_place()
            key = (data.get("name", ""), data.get("address", ""))

            if key in visited:
                print(f"↩️ 중복 스킵: {key[0]}")
            else:
                visited.add(key)
                results.append(data)
                print(f"✅ 수집: {data.get('name','')} / 리뷰 {len(data.get('reviews',[]))}개")

            # 다음 카드 클릭 위해 searchIframe 복귀
            switch_to_search_iframe()

        except Exception as e:
            print("⚠️ 카드 처리 중 에러:", repr(e))
            switch_to_search_iframe()
            continue

    # 4) 다음 페이지로 이동 (마지막 페이지 버튼 클릭)
    switch_to_search_iframe()
    moved = click_last_page_button()
    if not moved:
        print("🚫 더 이상 페이지 버튼 없음 → 전체 종료")
        break

    time.sleep(2)
    page_round += 1

########################################
# 결과 출력
########################################
print("\n========== 최종 결과 ==========")
print("총 가게 수:", len(results))
if results:
    print("첫 가게 예시:", results[0]["name"], "/", results[0]["category"], "/", results[0]["address"])
    print("첫 가게 리뷰 샘플:", results[0]["reviews"][:3])

# driver.quit()
